In [15]:
import os
import win32com.client
import numpy as np

###### define groups through which to access all utility functions
VHG_EinP = ['EinpA_mF','EinpA_oF','EinpF_mF','EinpF_oF','EinpK_mF','EinpK_oF','EinpU_mF','EinpU_oF']
Mode = ['bic']

VHG_main = [
"BSch_mF", "BSch_oF", "ERWmP_mF", "ERWmP_oF", "ERWoP_mF", "ERWoP_oF", "GSch_mF", "GSch_oF", "Kind_mF", "Kind_oF",
"NERWmP_mF", "NERWmP_oF", "NERWoP_mF", "NERWoP_oF", "Rentner_mF", "Rentner_oF", "Sch_mF", "Sch_ko", "Stud_mF", "Stud_oF"
]
Activity = ["A", "Ax", "B", "E", "F", "Fx", "G", "H", "J", "K", "Kx","S", "U", "Ux", "W", "Y"]

#Parameter values for C to try out (see Excel 003)
C = [['-0.9','-0.8','-0.7']]

def replace_C(s, new_number):
    parts = s.split('*', 1)
    sign = new_number[0]  # '+' or '-'
    if sign == '+' or sign == '-':
        index = parts[1].find("Matrix(5200)")
        if index != -1 and index > 0:
            parts[1] = parts[1][:index-1] + sign + parts[1][index:]
    return f"{new_number}*{parts[1]}"

distance_bins = [(-1, 0.5), (0.5, 1), (1, 2), (2, 5), (5, 10), (10, 20), (20, 50), (50, 9999)]
distance_bins_agg = [(-1, 1), (1, 2), (2, 5), (5, 10), (10, 20), (20, 9999)]

def print_trip_shares(trip_matrix, dis_matrix, lower_bound, upper_bound, total_trips):
        mask = (dis_matrix > lower_bound) & (dis_matrix <= upper_bound)
        trip_share = round(np.sum(trip_matrix[mask]) / total_trips *100)
        print(f"Anteil Fahrten {lower_bound} - {upper_bound} km: {trip_share}")

def print_trip_shares_bv(trip_matrix, dis_matrix, lower_bound, upper_bound, total_trips):
        distance_mask = (dis_matrix > lower_bound) & (dis_matrix <= upper_bound)
        mask = upper_left_mask & distance_mask
        trip_share = round(np.sum(trip_matrix[mask]) / total_trips *100)
        print(f"Anteil Fahrten {lower_bound} - {upper_bound} km: {trip_share}")

In [ ]:
#Loop over iteration steps to change C, rerun model, compute mode shares by distance classes, and save
for i in range(2,3):
    #verPath = r'D:\ver\20240709_1200.ver'
    verPath = r'C:\Users\arning.FUL\Desktop\ver\DEV_20250115_1325.ver'
    Visum = win32com.client.Dispatch('Visum.Visum.24')
    Visum.loadVersion(verPath)
    visem_einp = Visum.Procedures.Operations.ItemByKey(44)
    
    #change utils for Einpendler
    for mode in Mode:
        for vhg in VHG_EinP:
            
            #get old utilityfunction
            util = visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).AttValue("UtilityFunction")
            
            #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
            if util != "-99999":
                
                #get new C value based on iteration
                if mode == "bic":
                    new_C = C[0][i]
                else:
                    print("ERROR")                
            
                #write new utilityfunction
                new_util = replace_C(util,new_C)
                
                #write into Visum
                visem_einp.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupModeModeChoiceParameters(mode).SetAttValue("UtilityFunction", new_util)

    #change utils for main visem
    visem_main = Visum.Procedures.Operations.ItemByKey(45)
    for mode in Mode:
        for vhg in VHG_main:
            for activity in Activity:
                
                #get old utility function
                #not all combinations of vhg and activities exist; skip those exceptions
                try:
                    util = visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).AttValue("UtilityFunction")
                    #if utility isnt "-99999"... (which would mean that the respective bicycle isnt available for this person group)
                    if util != "-99999":

                        #get new C value based on iteration
                        if mode == "bic":
                            new_C = C[0][i]
                        else:
                            print("ERROR")

                        #write new utilityfunction
                        new_util = replace_C(util,new_C)

                        #write into Visum
                        visem_main.VISEMCombinedParameters.VISEMModeChoiceParameters.VISEMDGroupModeChoiceParameters(vhg).VISEMDGroupActivityModeModeChoiceParameters(activity,mode).SetAttValue("UtilityFunction", new_util)
                except Exception as e:
                    pass
                    # handle exception (skip the current iteration)
                    # print(f"Skipping vhg: {vhg}, activity: {activity}, mode: {mode} due to error: {e}")
    #Visum.Procedures.Execute()

    print("THIS IS FOR ITERATION NUMBER "+str(i))

    bic_trips_matrix = np.array(Visum.Net.Matrices.ItemByKey(51).GetValues())
    bic_dis_matrix = np.array(Visum.Net.Matrices.ItemByKey(5088).GetValues())
    upper_left_mask = np.zeros_like(bic_trips_matrix, dtype=bool)
    upper_left_mask[:357, :357] = True
    
    print('Bicycle BV')
    Anzbictrips = bic_trips_matrix[upper_left_mask].sum()
    for lower, upper in distance_bins_agg:
        print_trip_shares_bv(bic_trips_matrix, bic_dis_matrix, lower, upper, Anzbictrips)

    avbictriplength = round((bic_trips_matrix * bic_dis_matrix * upper_left_mask).sum()/Anzbictrips, 1)
    print('Bic av. trip length: '+ str(avbictriplength))

    #verPath_append = r'C:\Users\arning.FUL\Desktop\ver\DEV_20250115_0950_C1_'+str(i)+'.ver'
    #Visum.saveVersion(verPath_append)
    #Visum = None